In [125]:
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime

start = f"{str(datetime.now().year-20)}-01-01"
end = datetime.today().strftime('%Y-%m-%d')

print(start, end)

df_market = pd.read_csv("../data/Market_Stock_List.csv")

2005-01-01 2025-12-01


In [126]:
df_market.describe()

,Symbol,Company,Market,Industry,Sector,Address,Zip code,Tel.,Fax,Website
count,933,933,933,933,933,933,933,933,933,933
unique,933,933,2,9,28,920,109,888,734,908
top,ZIGA,ZIGA INNOVATION PUBLIC COMPANY LIMITED,SET,Property & Construction,-,"7-8th Floor, SCB Park Plaza 1, 18 Ratchadapise...",10120,-,-,-
freq,1,1,704,209,229,3,78,10,177,18


In [127]:
df_market.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 933 entries, 0 to 932
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Symbol    933 non-null    object
 1   Company   933 non-null    object
 2   Market    933 non-null    object
 3   Industry  933 non-null    object
 4   Sector    933 non-null    object
 5   Address   933 non-null    object
 6   Zip code  933 non-null    object
 7   Tel.      933 non-null    object
 8   Fax       933 non-null    object
 9   Website   933 non-null    object
dtypes: object(10)
memory usage: 73.0+ KB


In [128]:
df_market.nunique()

Symbol      933
Company     933
Market        2
Industry      9
Sector       28
Address     920
Zip code    109
Tel.        888
Fax         734
Website     908
dtype: int64

In [129]:
df_market["Industry"] = df_market["Industry"].replace('Industrials', 'Industrial')

In [130]:
df_ind = pd.DataFrame(df_market["Industry"].unique(), columns=["unique_values"])
df_ind.rename(columns={'unique_values': 'Industry'}, inplace=True)
df_ind['Symbol'] = None
print(df_ind.head() )

                  Industry Symbol
0               Industrial   None
1               Technology   None
2        Consumer Products   None
3  Property & Construction   None
4     Agro & Food Industry   None


In [131]:
df_all_ind = df_market.groupby("Industry")["Symbol"].apply(list).reset_index()

print(df_all_ind.head() )

                  Industry                                             Symbol
0     Agro & Food Industry  [AAI, APO, APURE, ASIAN, AU, BR, BRR, BTG, CBG...
1        Consumer Products  [88TH, AFC, AJA, APCO, AURA, BGT, BIZ, BLC, BT...
2               Financials  [ACAP, AEONTS, AF, AIRA, AMANAH, ASAP, ASK, AS...
3               Industrial  [2S, ACG, ADB, AH, AJ, AKR, ALLA, ALUCON, AMC,...
4  Property & Construction  [A, A5, AIMCG, AIMIRT, AKS, ALLY, AMATA, AMATA...


In [132]:
import yfinance as yf

for i,row in df_all_ind.iterrows():
    useable = []
    for tk in row['Symbol']:
        ticker = tk+".BK"
        data = yf.download(ticker, start=start, end=end, interval="1d", auto_adjust=True, progress=True)
        if len(data) < 2:
            continue
        data = data.dropna()
        data.index = pd.to_datetime(data.index)
        if (data.index[0].year != pd.to_datetime(start).year):
            continue
        if not data.empty:
            useable.append(tk)
    df_ind.at[i,'Symbol'] = useable
    

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [133]:
print(df_ind.head())

                  Industry                                             Symbol
0               Industrial  [APURE, ASIAN, BR, CFRESH, CHOTI, CM, CPF, CPI...
1               Technology  [AFC, BTNC, CPH, CPL, DTCI, FANCY, JCT, KYE, L...
2        Consumer Products  [ACAP, AEONTS, AF, AMANAH, ASK, ASP, AYUD, BAY...
3  Property & Construction  [AH, AJ, ALUCON, AMC, BCT, BSBM, CEN, CIG, CPR...
4     Agro & Food Industry  [A, A5, AKS, AMATA, AP, BLAND, CCP, CGD, CI, C...


In [134]:
# df is your original DataFrame with columns "head1" and "head2" (head2 contains lists)
expanded = pd.DataFrame(df_ind['Symbol'].tolist())                 # expand lists into columns
out = pd.concat([df_ind[['Industry']].reset_index(drop=True), expanded], axis=1)
out.columns = ['Industry'] + ['Symbol'] + [''] * (expanded.shape[1] - 1)  # header: head1, head2, , , ...
out = out.fillna('')                                          # missing -> blank/None in CSV
out.to_csv('out.csv', index=False)

In [135]:
ls = ['^AGRO.BK', '^CONSUMP.BK', '^FINCIAL.BK', '^INDUS.BK', '^PROPCON.BK', '^RESOURC.BK', '^SERVICE.BK', '^TECH.BK']


In [156]:
all_ind = 0
no_mc = 0
no_so = 0
no_ff = 0
ls_no_mc= []
ls_no_so= []
ls_no_ff= []
for i,row in df_ind.iterrows():
    all_ind += len(row['Symbol'])
    for tk in row['Symbol']:
        try:
            now_mc = yf.Ticker(tk+'.BK').info['marketCap']
        except:
            ls_no_mc.append(tk)
            no_mc += 1

        try:
            now_so = yf.Ticker(tk+'.BK').info['sharesOutstanding']
        except:
            ls_no_so.append(tk)
            no_so += 1

        try:
            now_ff = yf.Ticker(tk+'.BK').info['floatShares']
        except:
            ls_no_ff.append(tk)
            no_ff += 1


print(all_ind)
print("NO marketCap:", no_mc, ls_no_mc)
print("NO sharesOutstanding:", no_so, ls_no_so)
print("NO free_float:", no_ff, ls_no_ff)


361
NO marketCap: 2 ['S&J', 'WELL']
NO sharesOutstanding: 2 ['S&J', 'WELL']
NO free_float: 5 ['ACAP', 'MNIT', 'TAPAC', 'TIF1', 'MANRIN']


In [ ]:
cant_cal = ['S&J', 'WELL', 'ACAP', 'MNIT', 'TAPAC', 'TIF1', 'MANRIN']

In [160]:
print(yf.Ticker('APURE.BK').info['floatShares'] / yf.Ticker('APURE.BK').info['sharesOutstanding'])

0.17493926983997687
